In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"citrarahmawati","key":"bb626f6b077c83e40fd3a0f1948ed29b"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

kaggle.json


In [3]:
!kaggle datasets download -d andrewmvd/autism-screening-on-adults

Dataset URL: https://www.kaggle.com/datasets/andrewmvd/autism-screening-on-adults
License(s): other
  0% 0.00/7.38k [00:00<?, ?B/s]
100% 7.38k/7.38k [00:00<00:00, 14.3MB/s]


In [4]:
!unzip autism-screening-on-adults.zip -d autism-screening-on-adults
!ls autism-screening-on-adults

Archive:  autism-screening-on-adults.zip
  inflating: autism-screening-on-adults/autism_screening.csv  
autism_screening.csv


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score

import seaborn as sns
from keras.layers import Dense, BatchNormalization, Dropout, LSTM
from keras.models import Sequential
from keras.utils import to_categorical
from keras import callbacks

import tensorflow as tf
from tensorflow.keras.optimizers import Adam

In [6]:
df = pd.read_csv("/content/autism-screening-on-adults/autism_screening.csv")

In [7]:
df.head()

A1_Score  A2_Score  A3_Score  A4_Score  A5_Score  A6_Score  A7_Score  \
0         1         1         1         1         0         0         1   
1         1         1         0         1         0         0         0   
2         1         1         0         1         1         0         1   
3         1         1         0         1         0         0         1   
4         1         0         0         0         0         0         0   

   A8_Score  A9_Score  A10_Score  ...  gender       ethnicity jundice austim  \
0         1         0          0  ...       f  White-European      no     no   
1         1         0          1  ...       m          Latino      no    yes   
2         1         1          1  ...       m          Latino     yes    yes   
3         1         0          1  ...       f  White-European      no    yes   
4         1         0          0  ...       f               ?      no     no   

   contry_of_res used_app_before result     age_desc relation Class/ASD  
0  United States              no    6.0  18 and more     Self        NO  
1         Brazil              no    5.0  18 and more     Self        NO  
2          Spain              no    8.0  18 and more   Parent       YES  
3  United States              no    6.0  18 and more     Self        NO  
4          Egypt              no    2.0  18 and more        ?        NO  

[5 rows x 21 columns]

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 704 entries, 0 to 703
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   A1_Score         704 non-null    int64  
 1   A2_Score         704 non-null    int64  
 2   A3_Score         704 non-null    int64  
 3   A4_Score         704 non-null    int64  
 4   A5_Score         704 non-null    int64  
 5   A6_Score         704 non-null    int64  
 6   A7_Score         704 non-null    int64  
 7   A8_Score         704 non-null    int64  
 8   A9_Score         704 non-null    int64  
 9   A10_Score        704 non-null    int64  
 10  age              702 non-null    float64
 11  gender           704 non-null    object 
 12  ethnicity        704 non-null    object 
 13  jundice          704 non-null    object 
 14  austim           704 non-null    object 
 15  contry_of_res    704 non-null    object 
 16  used_app_before  704 non-null    object 
 17  result          

In [9]:
col_names = ['A1_Score','A2_Score','A3_Score','A4_Score','A5_Score','A6_Score','A7_Score','A8_Score','A9_Score','A10_Score','age','gender','ethnicity','jundice','austim','contry_of_res','used_app_before','result','age_desc','relation','Class/ASD']

for col in col_names:
    print (df[col].value_counts())

A1_Score
1    508
0    196
Name: count, dtype: int64
A2_Score
0    385
1    319
Name: count, dtype: int64
A3_Score
0    382
1    322
Name: count, dtype: int64
A4_Score
0    355
1    349
Name: count, dtype: int64
A5_Score
0    353
1    351
Name: count, dtype: int64
A6_Score
0    504
1    200
Name: count, dtype: int64
A7_Score
0    410
1    294
Name: count, dtype: int64
A8_Score
1    457
0    247
Name: count, dtype: int64
A9_Score
0    476
1    228
Name: count, dtype: int64
A10_Score
1    404
0    300
Name: count, dtype: int64
age
21.0     49
20.0     46
23.0     37
22.0     37
19.0     35
24.0     34
27.0     31
18.0     31
30.0     30
26.0     28
25.0     27
29.0     27
28.0     24
31.0     21
17.0     18
32.0     18
35.0     17
37.0     17
33.0     16
40.0     16
42.0     15
36.0     13
38.0     12
34.0     12
43.0     11
44.0     10
47.0      8
39.0      7
53.0      6
55.0      6
46.0      6
50.0      5
52.0      5
45.0      4
48.0      4
49.0      3
41.0      3
58.0      2
54.0     

In [10]:
numerical = []
catgcols =[]

for col in df.columns:
  if df[col].dtype=="float64":
    numerical.append(col)
  else:
    catgcols.append(col)

for col in df.columns:
  if col in numerical:
    df[col].fillna(df[col].median(), inplace=True)
  else:
    df[col].fillna(df[col].mode()[0], inplace=True)

In [11]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in catgcols:
    df[col] = le.fit_transform(df[col])

In [12]:
df.drop(columns=['A1_Score','A2_Score','A3_Score','A4_Score','A5_Score','A6_Score','A7_Score','A8_Score','A9_Score','A10_Score'], inplace=True)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 704 entries, 0 to 703
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   age              704 non-null    float64
 1   gender           704 non-null    int64  
 2   ethnicity        704 non-null    int64  
 3   jundice          704 non-null    int64  
 4   austim           704 non-null    int64  
 5   contry_of_res    704 non-null    int64  
 6   used_app_before  704 non-null    int64  
 7   result           704 non-null    float64
 8   age_desc         704 non-null    int64  
 9   relation         704 non-null    int64  
 10  Class/ASD        704 non-null    int64  
dtypes: float64(2), int64(9)
memory usage: 60.6 KB


In [14]:
import plotly.express as px
fig = px.histogram(df, x='Class/ASD')

fig.update_layout(title_text='<b>Count of Autistic and Non-Autistic Patients</b>', title_x=0.5)

fig.show()

In [15]:
X = df.drop("Class/ASD",axis=1)
Y = pd.get_dummies(df['Class/ASD'])

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25)

print(f"Shape of X_train is: {X_train.shape}")
print(f"Shape of Y_train is: {Y_train.shape}\n")
print(f"Shape of X_test is: {X_test.shape}")
print(f"Shape of Y_test is: {Y_test.shape}")

Shape of X_train is: (528, 10)
Shape of Y_train is: (528, 2)

Shape of X_test is: (176, 10)
Shape of Y_test is: (176, 2)


In [17]:
input_dim = X.shape[1]

In [18]:
model = Sequential()
model.add(Dense(8, input_dim = input_dim, kernel_initializer='normal', activation='relu'))
model.add(Dense(5, activation = "relu", kernel_initializer='normal'))
model.add(Dense(2, activation = 'sigmoid'))

In [19]:
model.compile(optimizer = Adam(learning_rate = 0.001),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 88        
                                                                 
 dense_1 (Dense)             (None, 5)                 45        
                                                                 
 dense_2 (Dense)             (None, 2)                 12        
                                                                 
Total params: 145 (580.00 Byte)
Trainable params: 145 (580.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
result = model.fit(X_train, Y_train, epochs = 15, batch_size = 10)

Epoch 1/15
53/53 [==============================] - 2s 5ms/step - loss: 0.6873 - accuracy: 0.7273
Epoch 2/15
53/53 [==============================] - 0s 5ms/step - loss: 0.6724 - accuracy: 0.8523
Epoch 3/15
53/53 [==============================] - 0s 4ms/step - loss: 0.6557 - accuracy: 0.8958
Epoch 4/15
53/53 [==============================] - 0s 7ms/step - loss: 0.6420 - accuracy: 0.8655
Epoch 5/15
53/53 [==============================] - 0s 8ms/step - loss: 0.6199 - accuracy: 0.8845
Epoch 6/15
53/53 [==============================] - 0s 7ms/step - loss: 0.5989 - accuracy: 0.9072
Epoch 7/15
53/53 [==============================] - 0s 7ms/step - loss: 0.5754 - accuracy: 0.8920
Epoch 8/15
53/53 [==============================] - 0s 7ms/step - loss: 0.5551 - accuracy: 0.8939
Epoch 9/15
53/53 [==============================] - 0s 7ms/step - loss: 0.5358 - accuracy: 0.8826
Epoch 10/15
53/53 [==============================] - 0s 9ms/step - loss: 0.5115 - accuracy: 0.8996
Epoch 11/15
53/53 [

In [22]:
y_pred =model.predict(X_test)
y_pred = (y_pred>0.5)

6/6 [==============================] - 0s 4ms/step


In [23]:
accuracy = accuracy_score(Y_test,y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.9147727272727273


In [24]:
model.save('./autism.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



In [25]:
model = tf.keras.models.load_model('./autism.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
open("./autism.tflite", "wb").write(tflite_model)

2644